In [13]:
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
import torch

In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch


In [15]:
dataset = load_dataset("imdb", split="train[:1%]").train_test_split(test_size=0.2)


In [16]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)


In [18]:
tokenized_datasets = dataset.map(tokenize, batched=True)


Map: 100%|██████████| 50/50 [00:00<00:00, 2368.11 examples/s]


In [19]:
tokenized_datasets.set_format("torch", columns=["input_ids", "attention_mask", "label"])


In [20]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=1,
    eval_strategy="epoch",
    logging_dir="./logs",
    report_to="none",
    no_cuda=True
)


/opt/miniconda3/envs/mloldlib/lib/python3.10/site-packages/transformers/training_args.py:1636: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [21]:

model.to("cpu")
tokenized_datasets.set_format("torch", columns=["input_ids","attention_mask","label"], device="cpu")

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"]
)


In [23]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.007065


TrainOutput(global_step=25, training_loss=0.10364768981933593, metrics={'train_runtime': 43.5353, 'train_samples_per_second': 4.594, 'train_steps_per_second': 0.574, 'total_flos': 6623369932800.0, 'train_loss': 0.10364768981933593, 'epoch': 1.0})

In [24]:
text = "This movie was awesome!"
inputs = tokenizer(text, return_tensors="pt")
pred = torch.argmax(model(**inputs).logits)
print("Sentiment:", "Positive" if pred == 1 else "Negative")


Sentiment: Negative
